In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from fake_useragent import UserAgent
from datetime import datetime
from tqdm.auto import tqdm

In [3]:
session = requests.session()
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

In [4]:
def get_reviews(url_suf, headers):
    url = 'https://www.restoran.ru' + url_suf
    req = session.get(url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    rev_boxes = soup.find_all('div', {"class": "review-wrap"})
    for box in rev_boxes:
        rate_circle = box.find('div', {'class':'review-rating'})
        rating = rate_circle.find('span').text
        if rating in ('1.0', '5.0'):
            review_place = box.find('span', {'class': 'review-text-full'})
            if review_place is None:
                review_place = box.find('span', {'class': 'review-text-preview'})
            if rating == '1.0':
                bad.append(review_place.text)
            else:
                good.append(review_place.text)
    next_page = soup.find('a', {'class': 'next icon-arrow-right'})
    if next_page is not None and not next_page['href'].startswith('javascript'):
        get_reviews(next_page['href'], headers)

In [5]:
def get_nth_page(page_number, headers):
    global good, bad
    url = 'https://www.restoran.ru/msk/catalog/restaurants/all/?page=' + str(page_number)
    req = session.get(url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    restaurants = soup.find_all('a', {'class': 'reviews-link'})
    for rest in restaurants:
        get_reviews(rest['href'], headers)

In [6]:
def run_all(n_pages, headers):
    for i in tqdm(range(n_pages)):
        get_nth_page(i+1, headers)

In [7]:
good = []
bad = []
run_all(10, headers)

In [8]:
from pymorphy2 import MorphAnalyzer
from string import punctuation
from nltk.tokenize import word_tokenize
from collections import Counter

In [9]:
def parse_revs(morph, revs):
    lemmas = []
    for text in revs:
        tokens = word_tokenize(text)
        for word in tokens:
            word = word.strip(punctuation).lower()
            if word.isalpha():
                lemmas.append(morph.parse(word)[0].normal_form)
    count = Counter(lemmas)
    count = count.most_common(int(len(count) * 0.9))
    com_words = set([word[0] for word in count])
    return com_words

In [10]:
morph = MorphAnalyzer()
bad_com = parse_revs(morph, bad[:int(len(bad)*0.8)])
good_com = parse_revs(morph, good[:int(len(good)*0.8)])

In [11]:
dist_bad = bad_com - good_com
dist_good = good_com - bad_com

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
def identify_review(morph, review):
    lemmas = []
    tokens = word_tokenize(review)
    for word in tokens:
        word = word.strip(punctuation).lower()
        if word.isalpha():
            lemmas.append(morph.parse(word)[0].normal_form)
    words = set(lemmas)
    goodness = len(dist_good & words)
    badness = len(dist_bad & words)
    return (goodness > badness)

In [14]:
test_data = list(zip(bad[int(len(bad)*0.8):], [0]*int(len(bad)*0.8)))
test_data.extend(list(zip(good[int(len(bad)*0.8):], [1]*int(len(good)*0.8))))
y_true = [r[1] for r in test_data]
y_pred = [identify_review(morph, r[0]) for r in test_data]
print(accuracy_score(y_true, y_pred))

0.9183965640658555


Во-первых, из-за дисбаланса классов в обучающей выборке для оценки качества модели лучше использовать не accuracy, а F-score (или просто посмотреть точность и полноту).

Во-вторых, можно привесить словам веса в соответствии с количеством их употреблений в хороших/плохих отзывах, и учитывать это при оценке.

В-третьих, нарицательное имя или прилагательное в кавычках - признак сарказма и, соответственно, негативного отзыва, это тоже можно учесть

В-четвертых (не NLP), если на сайте продавцы/владельцы заведения могут оставлять комментарии под отзывами, то с большей вероятностью комментарии (в особенности с извинениями) будут под плохими отзывами, это тоже можно проверить при парсинге (нвличие комментариев, а также слов типа "жаль" в них).